In [ ]:
import pandas as pd

#teams, teams2 데이터 병합
teams1 = pd.read_csv('../data/1819/england-premier-league-teams1-2018-to-2019-stats.csv')
teams2 = pd.read_csv('../data/1819/england-premier-league-teams2-2018-to-2019-stats.csv')
teams = pd.merge(teams1, teams2, on="team_name", suffixes=('_t1','_t2'))

#중복 컬럼 제거 : _t1과 _t2가 같은 값을 가지면 하나 삭제
cols_to_drop = []
for col in teams.columns:
    if col.endswith('_t1'):
        base = col[:-3]
        t2_col = base + '_t2'
        if t2_col in teams.columns:
            if teams[col].equals(teams[t2_col]):
                cols_to_drop.append(t2_col)

#중복된 _t2 컬럼 제거
teams_clean = teams.drop(columns=cols_to_drop)

#컬럼 이름 정리(_t1, _t2 제거)
teams_clean.columns = [c.replace('_t1','').replace('_t2','') for c in teams_clean.columns]

#결과 저장
teams_clean.to_csv("../data/1819/englad-premier-league-teams-2018-to-2019-stats.csv", index=False)

""" print(len(teams1.columns))
print(len(teams2.columns))
print(len(teams.columns)) """


293
442
734


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

matches = pd.read_csv('../data/1819/england-premier-league-matches-2018-to-2019-stats.csv')
teams = pd.read_csv('../data/1819/england-premier-league-teams-2018-to-2019-stats.csv')
players = pd.read_csv('../data/1819/england-premier-league-players-2018-to-2019-stats.csv')

#승무패 라벨 생성
def get_result(row):
    if row['home_team_goal_count'] > row['away_team_goal_count']:
        return 1 #홈팀 승
    elif row['home_team_goal_count'] < row['away_team_goal_count']:
        return 0 #어웨이 승
    else:
        return 2 #무승부
    
matches['match_result'] = matches.apply(get_result, axis=1)

#홈팀 시즌 통계 병합
home_teams = teams.add_prefix("home_team_")
matches = matches.merge(
    home_teams, how='left',
    left_on='home_team_name', right_on='home_team_team_name'
)
matches.drop(columns=['home_team_team_name'], inplace=True)

#어웨이 시즌 통계 병합
away_teams = teams.add_prefix('away_team_')
matches = matches.merge(
    away_teams, how="left",
    left_on="away_team_name", right_on="away_team_team_name"
)

#선수 평균 통계 구하기
player_avg = players.groupby("team_name").mean(numeric_only=True).reset_index()

#홈팀 선수 평균 병합
home_players = player_avg.add_prefix("home_player_")
matches = matches.merge(
    home_players, how="left",
    left_on="home_team_name", right_on="home_player_team_name"
)
matches.drop(columns=["home_player_team_name"], inplace=True)

#어웨이 선수 평균 병합
away_players = player_avg.add_prefix("away_player_")
matches = matches.merge(
    away_players, how="left",
    left_on="away_team_name", right_on="away_player_team_name"
)
matches.drop(columns=["away_player_team_name"], inplace=True)

# 이 부분 문자열 데이터 숫자로 바꾸고 수정 필요
numeric_cols = matches.select_dtypes(include="number").columns
exclude_cols = ["home_team_goal_count", "away_team_goal_count", "match_result"]
feature_cols = [col for col in numeric_cols if col not in exclude_cols]

X = matches[feature_cols].fillna(0)  # 결측값 처리
y = matches["match_result"]

#정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#학습, 테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#모델 정의
lr = LogisticRegression(max_iter=1000, multi_class='multinomial')
rf = RandomForestClassifier(n_estimators=50, random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', n_estimators=50)

#앙상블(soft voting)
voting_clf = VotingClassifier(
    estimators=[('lr', lr),('rf',rf),('xgb',xgb)],
    voting='soft'
)

voting_clf.fit(X_train, y_train)

y_pred = voting_clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy : {acc:.4f}")


FileNotFoundError: [Errno 2] No such file or directory: '../data/1819/england-premier-league-teams-2018-to-2019-stats.csv'